### Importing the needed libraries

In [1]:
import numpy as np
import cv2
import albumentations as A
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Creating the transformer for data augmentation

In [15]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
])

### Augmenting the data for better predictions

In [22]:
augmentation_path = './Data/Training/no_tumor'
output = './Data/Training/augmented_no_tumor'
for img in os.listdir(augmentation_path):
    image = cv2.imread(os.path.join(augmentation_path, img))
    augmented = transform(image=image)['image']
    output_img_path = os.path.join(output, f"{img.split('.')[0]}_aug.jpg")
    cv2.imwrite(output_img_path, augmented)
    cv2.imwrite(output + '/' + img, image)

### Creating the train and test labels and data

In [11]:
train_data = []
train_labels = []
test_data = []
test_labels = []

### Scaling the data and splitting it and resizing it

In [12]:
for dir in os.listdir("./Data"):
    path = "./Data/" + dir
    for dir2 in os.listdir(path):
        path2 = path + "/" + dir2
        for img in os.listdir(path2):
            image = cv2.imread(path2 + "/" + img)
            image = cv2.resize(image, (64, 64))
            image = image.flatten() / 255.0
            if dir == "Testing":
                test_data.append(image)
                test_labels.append(dir2)
            else:
                train_data.append(image)
                train_labels.append(dir2)

### Convert the data type from `list` to `numpy array`

In [13]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [14]:
print(len(train_data), len(train_labels) , len(test_data), len(test_labels))

3265 3265 394 394


In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

### Convert our labels to number for better handling

In [16]:
le = preprocessing.LabelEncoder()

le.fit(train_labels)
train_labels = le.transform(train_labels)
le.fit(test_labels)
test_labels = le.transform(test_labels)

In [8]:
train_labels[:5], test_labels[:5]

(array([0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0]))

### Feature extractor

In [9]:
def feature_extractor(dataset):
    x_train = dataset
    image_dataset_list = []
    for image in range(x_train.shape[0]):
        df = pd.DataFrame()
        input_img = x_train[image, :, :, :]
        img = input_img

        pixel_values = img.reshape(-1)
        df['pixel_values'] = pixel_values

        num = 1
        kernels = []
        for theta in range(2):
            theta = theta/ 4.0 * np.pi
            for sigma in (1, 3):
                lamda = np.pi/4
                gamma = 0.5
                gabor_label = 'gabor_' + str(num)
                ksize = 9
                kernel = cv2.getGaborKernel((ksize,ksize), sigma, theta, lamda, gamma)
                kernels.append(kernel)
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img
                num += 1
        image_dataset_list.append(df)
    image_dataset = pd.concat(image_dataset_list, ignore_index=True)
    return image_dataset

In [10]:
image_features = feature_extractor(train_data)
image_features_test = feature_extractor(test_data)
print(image_features.shape, image_features_test.shape)

IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed

### Reshaping to a vector for SVM

In [118]:
n_features = image_features.shape[1]
image_features = np.expand_dims(image_features, axis=0)
X_for_SVM = np.reshape(image_features, (train_data.shape[0], -1))

n_features_test = image_features_test.shape[1]
image_features_test = np.expand_dims(image_features_test, axis=0)
X_for_SVM_test = np.reshape(image_features_test, (test_data.shape[0], -1))

print(X_for_SVM.shape , X_for_SVM_test.shape)

(3265, 61440) (394, 61440)


### Testing

In [17]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC

pca = PCA(n_components=0.98)
pca_train = pca.fit_transform(xtrain)
pca_test = pca.transform(xtest)
svm = SVC()

svm.fit(xtrain, ytrain)
y_pred = svm.predict(xtest)
print(accuracy_score(ytest, y_pred))


0.8330781010719756


### GUI


In [18]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel
from PyQt5.QtGui import QIcon # import this to handle the icon
from PyQt5.QtGui import QFont
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Brain Image Classification")  # set the title of the app
        self.setGeometry(300, 300, 600, 400)               # set the X and Y width and height of the window
        self.setWindowIcon(QIcon('./img.png'))
        label = QLabel("classifier", self)
        label.setFont(QFont('Arial', 20))
        label.setGeometry(0, 0, 500, 500)
def main():
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()          # display a window
    sys.exit(app.exec_())  # make the window stay until we exit from it
if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
